# Process flow for End to end requirement gather process using openAI and Semantic kernel

![Alt text](image-1.png)

# Folder structure

Root
-- HistoricalInformation(Contains historical data of features ,description and acceptance criteria of all the previous project which can be used for getting description and acceptance crteria for similar requirement)

    -- Feature_data.csv (A CSV storage for past infomration this data can come from any database(SQL,congnitive search,mongoDB). We used csv for reference) 

-- Skills
    -- AzureDevops

        --FeatureDescription (semantic function for genrating feature descritption and acceptance crieria based on predefined template)

        --IdentifyFeature (semantic function for identify features\requirement in given text)

        --isRequirement (semantic function to check if the string contains requirement)

        --native_function.py (Not used)

-- Projectdata (this folder includes everything relates to specific project including base, as wele as processed information)

    --Project_information (Contains reference information which will used as in addition to the discussion transcript to identify requirement)

        --Customer_objective.txt (contains what customer\company wants to achive by this project)

        --NFR.txt (Non functional requirement for the project which supposed to be part of the project even if its not been discussed)

        --project_objective.txt (what the objective of project sponsor.)

    --Rawtxt (folder where you have to put all the requirement discussion transcripts)

    --Requirement_discussion (This folder would be used by script to store finalized sub transcript which is  <3000 char and contains requirements)
    
    --temp (Teamp folder used for storing all the transcripts which are genrated as part of stranscript splitting process)

--End-to-end-requirement-genration-in-AzureDevops.ipynb (Jypyter Notebook stiches everything togather)

![Alt text](image-7.png)

# Information gathering
The output of this steps is all the relevent information which can directly of indirectly contribute to project success.
- Get Delivery Objectives (store the information in Root\Projectdata\Project_information\Customer_objective.txt and project_objective.txt)
- Gather Non-functional requirements (store the information in Root\Projectdata\Project_information\NFR.txt)
- Requirements elaboration session transcripts (Store the transcripts inside Root\Projectdata\Rawtxt\)
Once you have this information you can start with document processing by performing below mentined steps


# Document processing
The objective of this steps is to reduce the size of documents to limited document which actual contains requirement information.
The output of this step is te get all the sub transcripts containg requirement in Root\Projectdata\Requirement_discussion\ folder.
- Read raw discussion document from "Rawtxt" folders.
- Breaking big discussion transcript into smaller transcripts of size 3000 character.
- Put them in temp folder
- Check if subpart of the discussion contains requirement or not.
- It contains requirement put that into "Requirement_discussion" folder
- Import historical feature data in memory
Covered in steps 1 to 7
![Alt text](image-8.png)

Once you have the list of document which contains requirement we can proceed further with feature identification.

# Requirement identification and refinment
The objective of this process is to identify and groom features and make them ready for importing in to Azure devops
The optput of this steps is finalized list of features and there details.
- Identify features from Proceed transcript
- remove duplicate
- Refine ,select or add new feature
- groom the selected features
- finalize feature details to be imported in Azure devops
Covered in steps 7 to 11
# Import in Azure devops
The objective of this steps is to import features in Azure devops with there respective metadata.
covered step 12 and 13




## Step 1 
Install all python libraries

In [ ]:
!python -m pip install azure-devops
!python -m pip install semantic-kernel==0.3.10.dev0


## Step 2 
Import Packages required and instantiate objects

In [2]:
from dotenv import dotenv_values
import semantic_kernel as sk
from semantic_kernel import ContextVariables # Context to store variables and Kernel to interact with the kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion ,AzureTextEmbedding # AI services
import pandas as pd
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_skills import WebSearchEngineSkill
import semantic_kernel as sk
from semantic_kernel.connectors.memory.azure_cognitive_search import ( AzureCognitiveSearchMemoryStore)
import subprocess
import csv
import os
import json
import shutil
kernel = sk.Kernel() # Create a kernel instance
kernel1 = sk.Kernel() # Create another kernel instance for not having semanitc function in the same kernel 

useAzureOpenAI = True
config = dotenv_values(".env")
AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
AZURE_OPENAI_EM_DEPLOYMENT_NAME = config["AZURE_OPENAI_EM_DEPLOYMENT_NAME"]
BING_API_KEY = config["BING_API_KEY"]
AZURE_COGNITIVE_SEARCH_ENDPOINT = config["AZURE_COGNITIVE_SEARCH_ENDPOINT"]
AZURE_COGNITIVE_SEARCH_ADMIN_KEY = config["AZURE_COGNITIVE_SEARCH_ADMIN_KEY"]
vector_size = 1536
# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_text_embedding_generation_service("ada",AzureTextEmbedding(deployment_name=AZURE_OPENAI_EM_DEPLOYMENT_NAME,endpoint=AZURE_OPENAI_ENDPOINT,api_key=AZURE_OPENAI_API_KEY))
    connector = BingConnector(api_key=BING_API_KEY)
    web_skill = kernel1.import_skill(WebSearchEngineSkill(connector), "WebSearch")
    search_async = web_skill["searchAsync"]    
    kernel1.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
    kernel1.import_skill(sk.core_skills.TextMemorySkill())
    COLLECTION_NAME = "historicalfeaturedata"
    context = kernel1.create_new_context()
    context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = COLLECTION_NAME
    context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8
    # connector = AzureCognitiveSearchMemoryStore(vector_size, AZURE_COGNITIVE_SEARCH_ENDPOINT, AZURE_COGNITIVE_SEARCH_ADMIN_KEY )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

## Step 3
Import the **Semantic functions**:Azuredevops


In [3]:
# note: using skills from the samples folder
plugins_directory = "./skills"
# Import the semantic functions
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureDevOps")

## Step 4 
Splitting raw transcripts to files with <3000 char and copy them to Root\Projectdata\temp\

In [4]:
baseprojectfoler=(r"C:\Users\vigarudi\Documents\code\Python\getfeatures\Projectdata")
rawtxtfolder=feature_list=os.path.join(baseprojectfoler , "Rawtxt")
Requirement_folder_path = os.path.join(baseprojectfoler , "Requirement_discussion")
temp_folder_path = os.path.join(baseprojectfoler , "temp")
# Delete the contents of the folder
if os.path.exists(temp_folder_path):
    shutil.rmtree(temp_folder_path)
os.mkdir(temp_folder_path)
max_size = 3000
file_list = os.listdir(rawtxtfolder )
x=0
# Iterate through each file and read its contents
for filename in file_list:
    file_path = os.path.join(rawtxtfolder , filename)  
    targetfilepath=os.path.join(temp_folder_path , filename)  
    infoid="info"+str(x)
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            content = file.read()
            num_parts = (len(content) + max_size - 1) // max_size
            for i in range(num_parts):
                part = content[i*max_size:(i+1)*max_size]
                part_path = f"{targetfilepath}_{i+1}.txt"
                with open(part_path, "w") as part_file:
                    part_file.write(part)
    x=x+1


## Step 5
Identify splitted files which are actually containing requirement using "isrequirement" sementic function and copy them to Root\Projectdata\Requirement_discussion\ if they do have requirement.

In [ ]:
FIdenFunction = DevFunctions["isrequirement"] 
max_size = 3000
file_list = os.listdir(temp_folder_path)
x=0
# Iterate through each file and read its contents
for filename in file_list:
    file_path = os.path.join(temp_folder_path , filename)  
    infoid="info"+str(x)
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contentf = file.read()
            resultFD = FIdenFunction(contentf)
            print(file_path +"   "+ str(resultFD))
            if "yes" in str(resultFD).lower() :
                shutil.copy(file_path, Requirement_folder_path)

## Step 6
Read all project information and set project folders.

In [5]:
Requirement_folder_path = (r"C:\Users\vigarudi\Documents\code\Python\getfeatures\Projectdata\Requirement_discussion")  # Replace with the path to your folder
Project_folder_path = (r"C:\Users\vigarudi\Documents\code\Python\getfeatures\Projectdata\Project_information")  # Replace with the path to your folder
feature_list=os.path.join(Project_folder_path , "featurelist_raw.json")
Risk_list=os.path.join(Project_folder_path , "Risklist_raw.json")
CustomerObjective= os.path.join(Requirement_folder_path, "Customer_objective.txt") # Replace with the path to your folder
if os.path.isfile(CustomerObjective):
        with open(CustomerObjective, 'r') as file:
            #context["cprojectgoal"]  = file.read()
            context["cprojectgoal"]  = ""
ISD_objective= os.path.join(Requirement_folder_path, "ISD_objective.txt") # Replace with the path to your folder
if os.path.isfile(ISD_objective):
        with open(ISD_objective, 'r') as file:
            #context["iprojectgoal"]  = file.read()
            context["iprojectgoal"]  = ""
NFR= os.path.join(Requirement_folder_path, "NFR.txt") # Replace with the path to your folder
if os.path.isfile(NFR):
        with open(NFR, 'r') as file:
            context["nfrtext"] = ""
            #context["nfrtext"] = file.read()
# List all files in the folder


## Step 7 
Import historical data in memory for using it subsequent steps. If we are using external source such as SQL,COgnitive search, mongodb then this steps would change according to source.
![Alt text](image-2.png)

![Alt text](image.png)

In [ ]:
import csv
kernel1.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel1.import_skill(sk.core_skills.TextMemorySkill())
csv_file_path=r"C:\Users\vigarudi\Documents\code\Python\getfeatures\HistoricalInformation\Feature_data.csv"
with open(csv_file_path, mode='r', newline='') as file:
            reader = csv.reader(file)
            x=0
            # Loop through each row in the CSV file
            for row in reader:
            # Each row is a list of values
            # Access values by index, e.g., row[0], row[1], ...        
                if row[0] != "featurename":
                    if row[2] is None or row[2] == ""or row[2] == " ":
                        row[2]="none"
                    if row[1] is None or row[2] == "" or row[2] == " ":
                        row[1]="none"
                    txq='{"featurename":"' +row[0]+ '", "description":"' +row[1] + '","acceptancecriteria":"' + row[2] + '"}'
                    infoid="info"+str(x)
                    print(txq)
                    await kernel1.memory.save_information_async(COLLECTION_NAME , id=infoid, text=txq)
                x=x+1   

## Step 8 
We pick files from Root\Projectdata\Requirement_discussion\ folder and process those using "IdentifyFeatures" semantic function.Post that we would remove dulicate requirement.

In [ ]:
allfeatures=[]
Requirement_folder_path = (r"C:\Users\vigarudi\Documents\code\Python\getfeatures\Projectdata\Requirement_discussion")  # Replace with the path to your folder
allfeatures=[]
FIdenFunction = DevFunctions["IdentifyFeatures"] 
file_list = os.listdir(Requirement_folder_path )
x=1
# Iterate through each file and read its contents
for filename in file_list:
    file_path = os.path.join(Requirement_folder_path , filename)    
    infoid="info"+str(x)
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            context["requirementtext"] = file.read()
            if len(allfeatures ) != 0:
                #context["featurelist"] = str(allfeatures)
                context["featurelist"] = ""
            resultIF = FIdenFunction(context=context)
            print(resultIF)
            jsonIF = json.loads(str(resultIF))
            print(jsonIF)
            for feature in jsonIF:
                allfeatures.append({"featureid": x,"featurename": feature["feature"],"descriptions":[{"descriptionid": 1,"description":feature["description"],"acceptancecriteria":"","approved":"n","source":"Identify feature"}],"discussionname":file_path,"approved":"n"})
                x=x+1
            # Now, you can work with the file_contents as a string
            #print(filename , file_contents)
            #await kernel1.memory.save_information_async(COLLECTION_NAME, id=infoid, text=context["requirementtext"])          

#Remove duplicate requirement
unique_array = []
names = set()
for obj in allfeatures:
    name = obj.get("featurename")
    if name is not None and name.lower() not in names:
        unique_array.append(obj)
        names.add(name.lower())

with open(feature_list, "w") as f:
    json.dump(unique_array, f)
          

## Step 9
We would perform manual refinment of requirement by updating details in base feature list Json "\Projectdata\Project_information\featurelist_raw.json" In this step we can update feature title ,description and select the feature which are required to implement project by changing the value of "approved" tag to "y". We can manually add feature if its been missed in list.

![Alt text](image-3.png)

## Step 10

Once you have finalized the list of feature, we groom these feature using "FeatureDescription" semantic function and historical data from memory.The outcome of this step would be, addition of multiple option of description and acceptance criteria.


In [ ]:
allfeatures=[]
FDesFunction = DevFunctions["FeatureDescription"] 
# Iterate through each file and read its contents
with open(feature_list, "r") as f:
    allfeatures1 = json.load(f)
for feature in allfeatures1:
    print(feature["approved"] +"  "+ feature["featurename"])
    if feature["approved"] == "y" or feature["approved"] == "Y":
            context["title"] =feature["featurename"]
            context["description"] =feature["descriptions"][0]["description"]
            print(context["description"])
            print(context["title"])
            resultFD = FDesFunction(context=context)
            print(resultFD)
            jsonIF = json.loads(str(resultFD))
            feature["descriptions"].append({"descriptionid": 2,"description":jsonIF["description"],"acceptancecriteria":jsonIF["acceptancecriteria"],"approved":"y","source":"FeatureDescription"})
            # add historical description
            historicfeature= await kernel1.memory.search_async(COLLECTION_NAME, feature["featurename"])
            txtquery=historicfeature[0].text
            if historicfeature is not None:
                x=3
                for featuredata in historicfeature:
                    print(featuredata.text)
                    hsjson = json.loads(featuredata.text)
                    print(hsjson["featurename"])
                    feature["descriptions"].append({"descriptionid": x,"description":hsjson["description"],"acceptancecriteria":hsjson["acceptancecriteria"],"approved":"n","source":"historicaldata"})
                    x=x+1
            allfeatures.append({"featurename": jsonIF["featurename"],"descriptions":feature["descriptions"],"discussionname":feature["discussionname"],"approved":feature["approved"]})
    else:
        allfeatures.append(feature)
              # Now, you can work with the file_contents as a string
            #print(filename , file_contents)
            #await kernel1.memory.save_information_async(COLLECTION_NAME, id=infoid, text=context["requirementtext"])
            
    x=x+1       
    

with open(feature_list, "w") as f:
    json.dump(allfeatures, f)
          

## Step 11
This is the final step befor we import these features in Azure DevOps. In this steps,wWe would perform manual refinment of requirement by updating details in feature list Json "\Projectdata\Project_information\featurelist_raw.json". In this step, we can update feature title ,description and select the Description and acceptance criteria from "descriptions" array by changing the value of "approved" tag to "y". We can manually add\update feature description if its been missed in list.

![Alt text](image-4.png)

## Step 12

Once we have finalized list of features,description ,acceptance crieria(you can inlude other parameters such as efforts business value priority etc as well). we can extecute below step to import them in Azure devops.

In [6]:
from azure.devops.v7_1.py_pi_api import JsonPatchOperation
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
from azure.devops.v7_1.work_item_tracking.models import Wiql
import base64
from semantic_kernel import ContextVariables, Kernel
import re
import time
feature_title = "test"
description="description"
acceptancecriteria=""
targetOrganizationName= "vigarudi0944"
targetProjectName= "test"
#targetOrganizationPAT = "i3i2vw5scgscy6hn26wswrdjjowmicnrzlyi7gy5o2doofto2vsa"
targetOrganizationPAT = "cdleu3tixtj5i4kg27yy7dnm3sdmsjxjhfy5aeh3wpk5kkvjowpa"
teamName = "test Team"
areaName = teamName
iterationName ="Sprint 1"
targetOrganizationUri='https://dev.azure.com/'+targetOrganizationName
credentials = BasicAuthentication('', targetOrganizationPAT)
connection = Connection(base_url=targetOrganizationUri, creds=credentials)
userToken = "" + ":" + targetOrganizationPAT
base64UserToken = base64.b64encode(userToken.encode()).decode()
headers = {'Authorization': 'Basic' + base64UserToken}
core_client = connection.clients.get_core_client()
targetProjectId = core_client.get_project(targetProjectName).id

Lets import it in Azure devops.

In [7]:
import html
with open(feature_list, "r") as f:
    allfeatures = json.load(f)
for feature in allfeatures:
    if feature["approved"] == "y" or feature["approved"] == "Y":
        for description1 in feature["descriptions"]:
            if description1["approved"] == "y" or description1["approved"] == "Y":
                description=description1["description"]     
                acceptancecriteria=description1["acceptancecriteria"]   
        description = description.replace("\n", "<br>")
        acceptancecriteria = acceptancecriteria.replace("\n", "<br>")
        #print(description.encode('utf-8'))
        
        workItemObjects = [
        {
            'op': 'add',
            'path': '/fields/System.WorkItemType',
            'value': "Feature"
        },
        {
            'op': 'add',
            'path': '/fields/System.Title',
            'value': feature["featurename"]
        },
        {
            'op': 'add',
            'path': '/fields/System.State',
            'value': "New"
        },
        {
            'op': 'add',
            'path': '/fields/System.Description',
            'value': description
        },
        {
            'op': 'add',
            'path': '/fields/Microsoft.VSTS.Common.AcceptanceCriteria',
            'value': acceptancecriteria
        },      
        {
            'op': 'add',
            'path': '/fields/System.IterationPath',
            'value': targetProjectName+"\\"+iterationName
        }
            ]
        #for feature in work_client:
        #    if feature['path'] == '/fields/System.Description':
        #        print(feature['value'])            
        jsonPatchList = JsonPatchOperation(workItemObjects)
        print(targetProjectId)
        work_client = connection.clients.get_work_item_tracking_client()   
        max_attempts = 10  # Maximum number of attempts
        current_attempt = 0
        while current_attempt < max_attempts:
            time.sleep(5)
            errorvar=None
            try:
                Response = work_client.create_work_item(jsonPatchList.from_, targetProjectName, "Feature")
                print(Response)
            except Exception as e:
                errorvar=str(e)
                print(errorvar)
            if errorvar is None:
                break 
            current_attempt += 1

49cae7e9-144b-481c-afbb-bc6b5260c3a1
Error occurred in request., ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error occurred in request., ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
{'additional_properties': {}, 'url': 'https://dev.azure.com/vigarudi0944/49cae7e9-144b-481c-afbb-bc6b5260c3a1/_apis/wit/workItems/247', '_links': <azure.devops.v7_0.work_item_tracking.models.ReferenceLinks object at 0x000001E77D482650>, 'comment_version_ref': None, 'fields': {'System.AreaPath': 'test', 'System.TeamProject': 'test', 'System.IterationPath': 'test\\Sprint 1', 'System.WorkItemType': 'Feature', 'System.State': 'New', 'System.Reason': 'New feature', 'System.CreatedDate': '2023-10-25T22:30:03.34Z', 'System.CreatedBy': {'displayName': 'vivek', 'url': 'https://spsprodcus5.vssps.visual

![Alt text](image-6.png)
![Alt text](image-5.png)

In [ ]:
print(allfeatures)